In [1]:
spark

In [2]:
from pyspark import SparkConf

In [3]:
spark.conf.set('spark.executor.cores', 34)
spark.conf.set('spark.driver.cores', 2)
spark.conf.set('spark.driver.memory', '5g')
spark.conf.set('spark.executor.memory', '55g')
spark.conf.set('spark.cores.max', 34)
spark.conf.set('spark.default.parallelism', 34)

In [4]:
# SparkConf()
# conf = SparkConf()
# # conf.set("spark.storage.memoryFraction", "0.5")
# sc = SparkContext(conf=conf)

In [5]:
import pyspark
from pyspark import SQLContext
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import TimestampType

import warnings
warnings.filterwarnings(action='ignore')

import pandas as pd
import numpy as np
import pandas as pd
from sklearn import *
from sklearn import preprocessing
# import lightgbm as lgb
from multiprocessing import *
from sklearn.model_selection import StratifiedKFold

pd.set_option('display.max_columns', None)

# sc = pyspark.SparkContext('local[*]')
sc=spark
sqlContext = SQLContext(sparkContext=sc)

#sc._conf.getAll()

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/opt/conda/lib/python3.6/site-packages/sklearn/lda.py:6: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
/opt/conda/lib/python3.6/site-pac

In [6]:
s3_path = "/home/jovyan/work/sparkling-water-2.2.2/uwork/s3/flight.pq.11"
out_path = "/home/jovyan/work/sparkling-water-2.2.2/work/s3.flight.pq.11.comb"

In [7]:
!pwd

/home/jovyan/work/sparkling-water-2.2.2/work/flight/prod


In [8]:
! ls /home/jovyan/work/sparkling-water-2.2.2/uwork/s3/flight.pq.11

flight_10_1   flight_13_35  flight_22_11  flight_33_6	flight_6_18
flight_10_14  flight_13_36  flight_22_12  flight_33_7	flight_6_19
flight_10_15  flight_13_37  flight_22_13  flight_33_8	flight_6_2
flight_10_16  flight_13_38  flight_22_14  flight_33_9	flight_6_20
flight_10_17  flight_13_39  flight_22_5   flight_34_10	flight_6_21
flight_10_18  flight_13_4   flight_22_6   flight_34_11	flight_6_22
flight_10_19  flight_13_40  flight_22_7   flight_34_12	flight_6_23
flight_10_2   flight_13_9   flight_22_8   flight_34_13	flight_6_24
flight_10_20  flight_14_1   flight_22_9   flight_34_14	flight_6_25
flight_10_21  flight_14_10  flight_23_10  flight_34_5	flight_6_26
flight_10_22  flight_14_11  flight_23_11  flight_34_6	flight_6_27
flight_10_23  flight_14_12  flight_23_12  flight_34_7	flight_6_28
flight_10_24  flight_14_13  flight_23_13  flight_34_8	flight_6_29
flight_10_25  flight_14_15  flight_23_14  flight_34_9	flight_6_3
flight_10_26  flight_14_16  flight_23_5   flight_3_5	flight_6_30
flight_1

In [9]:
! ls /home/jovyan/work/sparkling-water-2.2.2/work/s3.flight.pq.11.comb

ls: cannot access '/home/jovyan/work/sparkling-water-2.2.2/work/s3.flight.pq.11.comb': No such file or directory


In [10]:
from glob import glob

folders = glob(s3_path+"/*/")

try:
    folders.remove(s3_path+"/pq/") #pq/flight_10_31 Is this duplicate?
except ValueError:
    print("error")

In [12]:
len(folders)

622

In [ ]:
%%time

sample_prop = 1

import time

start = time.time()

for i, f in enumerate(folders):
    end = time.time()
    elapsed_min = (end - start)/60
    print("Processing: " + str(i) + "/" + str(len(folders)) + "  " + 
          f.replace(s3_path, "").replace("/", "") + " Elapsed time: " + str(round(elapsed_min, 0)) + " minutes")
    
    if (sample_prop == 1): #no sample
        df = (sqlContext.read.parquet(f)          
                .withColumn('searchMonth', F.date_format(F.last_day(F.col('searchDate')), "YYYYMM")))    
    else:    
        df = (sqlContext.read.parquet(f)
                .sample(False, sample_prop, seed=0)
                .withColumn('searchMonth', F.date_format(F.last_day(F.col('searchDate')), "YYYYMM")))    
        
    if (i == 0):        
        df.coalesce(2).write.partitionBy("fromCity", "toCity", "searchMonth").mode("overwrite").save(out_path)
    else:        
        df.coalesce(2).write.partitionBy("fromCity", "toCity", "searchMonth").mode("append").save(out_path)            

Processing: 0/622  flight_13_16 Elapsed time: 0.0 minutes
Processing: 1/622  flight_23_14 Elapsed time: 1.0 minutes


In [ ]:
df = sqlContext.read.parquet(out_path)

In [ ]:
df.count()

In [ ]:
lookforwardDays = 19
ndays = lambda i: i * 86400 # number of seconds in a day
byVar = ['toCity', 'depDate', 'stayDays', 'timeline1_departureTime_time', 'timeline2_departureTime_time',
         'timeline1_carrier_airlineCode', 'timeline2_carrier_airlineCode']

w = (Window.partitionBy('routeCombKey')
     .orderBy(F.col('searchDate').cast('timestamp').cast('long'))
     .rowsBetween(0, ndays(lookforwardDays)))

threshold = 20

udfGetWeekNumer = lambda dt: F.udf(dt.isocalendar()[1], IntegerType())
# udfGetAvg = lambda x: F.udf(sum(x)/float(len(x)))\n\n

df2 = (df.filter(df.price > 0)
       .withColumn('routeCombKey', F.concat_ws('-', F.col('fromCity'),
                                   F.col('toCity'),
                                   F.col('stayDays'), F.col('depDate'),
                                   F.col('timeline1_departureTime_time'),
                                   F.col('timeline2_departureTime_time'),
                                   F.col('timeline1_carrier_airlineCode'),
                                   F.col('timeline2_carrier_airlineCode')))
                                   # join_udf(F.col('timeline1_departureTime_time')),
                                   # join_udf(F.col('timeline2_departureTime_time')),
                                   # join_udf(F.col('timeline1_carrier_airlineCode')),
                                   # join_udf(F.col('timeline2_carrier_airlineCode'))))
       .withColumn('futureMinPrice', F.min(F.col('price')).over(w))
       .withColumn('priceWillDrop', (F.col('price') - F.col('futureMinPrice') > threshold).cast('int'))
       .withColumn('saving0', F.col('price') - F.col('futureMinPrice'))
       .withColumn('temp0', F.lit(0))
       .withColumn('saving', F.greatest('saving0', 'temp0'))
       .drop('saving0', 'temp0')
       .withColumn('leadTime', F.datediff(F.col('depDate'), F.col('searchDate')))
       .withColumn('depWeekOfYear', F.weekofyear(F.col('depDate')))
       .withColumn('retWeekOfYear', F.weekofyear(F.col('leg2_departureTime_date')))
       .withColumn('depMonth', F.month(F.col('depDate')))
       .withColumn('depDayOfMonth', F.dayofmonth(F.col('depDate')))
       .withColumn('depDayOfYear', F.dayofyear(F.col('depDate')))
       .withColumn('depDayOfWeek', F.date_format(F.col('depDate'), 'EEEE'))
       .withColumn('retDayOfWeek', F.date_format(F.col('leg2_departureTime_date'), 'EEEE'))
       .withColumn('searchDayOfWeek', F.date_format(F.col('searchDate'), 'EEEE'))
       .withColumn('leg1_noOfTicketsLeft', 
                   F.when(F.col('leg1_carrierSummary_noOfTicketsLeft') > 0, 
                          F.col('leg1_carrierSummary_noOfTicketsLeft')).otherwise(99))
        .withColumn('leg2_noOfTicketsLeft', 
                    F.when(F.col('leg2_carrierSummary_noOfTicketsLeft') > 0, 
                           F.col('leg2_carrierSummary_noOfTicketsLeft')).otherwise(99))
       .withColumn('leg1_cabinClass_0', F.col('timeline1_carrier_cabinClass').getItem(0))
       .withColumn('leg1_cabinClass_1', F.col('timeline1_carrier_cabinClass').getItem(1))
       .withColumn('leg1_cabinClass_2', F.col('timeline1_carrier_cabinClass').getItem(2))
       .withColumn('leg2_cabinClass_0', F.col('timeline2_carrier_cabinClass').getItem(0))
       .withColumn('leg2_cabinClass_1', F.col('timeline2_carrier_cabinClass').getItem(1))
       .withColumn('leg2_cabinClass_2', F.col('timeline2_carrier_cabinClass').getItem(2))
       .select('price', 'priceWillDrop', 'futureMinPrice', 'saving',
               'fromCity', 'toCity',
               'searchDate',
               'routeCombKey',
               'leadTime', 
               'leg1_stops', 'leg2_stops',
               'leg1_noOfTicketsLeft', 'leg2_noOfTicketsLeft',
               'leg1_carrierSummary_airlineName', 'leg2_carrierSummary_airlineName',
               'leg1_departureTime_hour', 'leg2_departureTime_hour',
               'depWeekOfYear', 'depDayOfWeek','retWeekOfYear', 'retDayOfWeek',
               'searchDayOfWeek',
               'leg1_cabinClass_0', 'leg1_cabinClass_1', 'leg1_cabinClass_2',
               'leg2_cabinClass_0', 'leg2_cabinClass_1', 'leg2_cabinClass_2',
               'trip')
      )

# df2.cache()

In [ ]:
%%time
# Slow!

df3 = df2.filter(F.col('searchDate') <= F.lit('2017-06-23').cast(TimestampType()))
df3 = df3.withColumn('randVar', F.round(F.rand()*200, 0))
# df3.filter(F.col('searchDate') >= F.lit('2017-06-16').cast(TimestampType())).toPandas().to_pickle("D:\flight.pq.11\df3_test.pkl")
# df3.groupby('randVar').agg(F.count('randVar').alias('count')).show(100)

# listRand = [x['randVar'] for x in df3.select('randVar').distinct().collect()]

# dfArray = [df3.where(F.col('randVar') == x) for x in listRand]

# # spark.conf.set("spark.driver.maxResultSize", "10g")

# i = 0

# for df_temp in dfArray:
#     df_temp.toPandas().to_pickle('../out_data/flight.pq.11_v2_pickle/df3_test_' + str(listRand[i]) + '.pkl')
#     i += 1

In [ ]:
%%time

out_path2 = "/home/jovyan/work/sparkling-water-2.2.2/work/s3.flight.pq.11.modeldata01"

df3.write.partitionBy("fromCity", "toCity").mode("overwrite").save(out_path2)